# Absenteeism Data Analysis for a Company

## Business Problem

The business environment of today is more competitive that it used to be which leads to increased pressure in the workplace. Therefore, it is reasonable to expect that unachievable business goals and an elevated risk of being unemployed can raise people's stress levels. Often, the continuous presence of such factors becomes detrimental to a person's health. Sometimes this may result in minor illness which of course is not desired. However, it may happen that the employee develops a longterm condition (i.e. depression).

That being said, we can take a look at the problem from the point of view of predicting absenteeism from work. Absenteeism can be defined as the absence from work during normal working hours resulting in temporary incapacity to execute regular working activity. We want to explore whether a person presenting certain characteristics is expected to be away from work at some point in time or not. In other words, we want to know for how many working hours an employee could be away from work based on information such as how far they live from their workplace, how many children and pets they have, their level of higher education, etc.

Having such information in advance can improve our decision making by showing us how we can reorganize the work process in a way that will allow us to avoid a lack of productivity and increase the quality of work generated in the firm.

We'll take a look at dataset from a preexisting study about the prediction of absenteeism at work. Each row of this data table describes an instance of an employee being absent from work for a certain number of hours during working hours.

## Import the relevant libraries

In [1]:
# Import the relevant libraries
import pandas as pd
import numpy as np

## Load the data

In [45]:
# Load and take a look at the raw csv data
raw_csv_data = pd.read_csv('Absenteeism-data.csv' )
raw_csv_data

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
695,17,10,23/05/2018,179,22,40,237.656,22,2,2,0,8
696,28,6,23/05/2018,225,26,28,237.656,24,1,1,2,3
697,18,10,24/05/2018,330,16,28,237.656,25,2,0,0,8
698,25,23,24/05/2018,235,16,32,237.656,25,3,0,0,2


- Absenteeism Time in Hours will be our dependent variable
- All other columns will be our independent variables

In [4]:
df = raw_csv_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
ID                           700 non-null int64
Reason for Absence           700 non-null int64
Date                         700 non-null object
Transportation Expense       700 non-null int64
Distance to Work             700 non-null int64
Age                          700 non-null int64
Daily Work Load Average      700 non-null float64
Body Mass Index              700 non-null int64
Education                    700 non-null int64
Children                     700 non-null int64
Pets                         700 non-null int64
Absenteeism Time in Hours    700 non-null int64
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


- Our dataframe doesn't seem to contain missing values.

## Data Preprocessing

### Drop 'ID' column
The ID is just a label variable and therefore will not help us predict the absenteeism of an individual.

In [5]:
df = df.drop(['ID'], axis=1)

### 'Reason for Absence'

In [6]:
df['Reason for Absence'].min()

0

In [7]:
df['Reason for Absence'].max()

28

We want to extract a list containing distinct values only.

In [8]:
df['Reason for Absence'].unique()

array([26,  0, 23,  7, 22, 19,  1, 11, 14, 21, 10, 13, 28, 18, 25, 24,  6,
       27, 17,  8, 12,  5,  9, 15,  4,  3,  2, 16])

In [9]:
len(df['Reason for Absence'].unique())

28

There should be 29 values in the range from 0 to 28, inclusive. Therefore, one value is missing.

In [10]:
sorted(df['Reason for Absence'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

We can see that the value 20 is missing.

These 28 reasons represent actual reasons why someone would not be able to come into work (i.e. Infectious disease, pregnancy, blood donation, etc.) Therefore, these are categorical nominal values and we need to add numeric meaning to them for analysis.

We can turn them into dummy variables. In this case, we will assume that an individual has been absent from work because of one, and only one, particular reason. We will check this assumption.

In [11]:
reason_columns = pd.get_dummies(df['Reason for Absence'])
reason_columns

,0,1,2,3,4,5,6,7,8,9,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
reason_columns['check'] = reason_columns.sum(axis=1)
reason_columns

,0,1,2,3,4,5,6,7,8,9,...,19,21,22,23,24,25,26,27,28,check
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
696,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [13]:
reason_columns['check'].unique()

array([1])

Since we have a value of 1 in each row of the 'check' column, we can be confident that each original data entry has exactly one reason for absence.

In [14]:
reason_columns = reason_columns.drop(['check'], axis=1)

To avoid issues with multicolinearity, we must remove one column with dummy variables. To preserve the logic of our analysis, this will be the column that stands for reason 0 (someone who has been away due to an unknown reason). This is because, if all of the rest of the reasons are 0, then we already know that the person was away for an unknown reason.

In [15]:
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first=True)
reason_columns

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


28 columns, one for each separate reason, is a lot for the analysis. In our case, it might be a good idea to group them according to a common characteristic (classify the various reasons for absence). We are able to do this because we confirmed that each data entry had exactly 1 reason for absence.

- Group 1: Reasons 1-14 are all related to various diseases.
- Group 2: Reasons 15-17 are all related to pregnancy and giving birth.
- Group 3: Reasons 18-21 are all related to poisoning or signs not elsewhere categorized.
- Group 4: Reasons 22-24 are all relatively light reasons for absence (i.e. blood donation).

For each group, we will look at the max value amongst its respective columns for each row. Therefore, if any of those columns were the reason for absence, the max value will show up as 1. If none of the columns in that group were the reason for absence, this will show up as 0, which is what we want.

In [16]:
reason_type_1 = reason_columns.loc[:,1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:,15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:,18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:,22:].max(axis=1)

In [17]:
# Let's add these columns back into the original df
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis=1)
# We will also drop the original 'Reason for Absence' column so we don't introduce multicolinearity
df = df.drop(['Reason for Absence'], axis = 1)
df

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,23/05/2018,179,22,40,237.656,22,2,2,0,8,1,0,0,0
696,23/05/2018,225,26,28,237.656,24,1,1,2,3,1,0,0,0
697,24/05/2018,330,16,28,237.656,25,2,0,0,8,1,0,0,0
698,24/05/2018,235,16,32,237.656,25,3,0,0,2,0,0,0,1


These new column names have no meaning, therefore we want to give them more appropriate names and also reorder the columns in the dataframe.

In [19]:
df.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [27]:
column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

df.columns = column_names
df.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason_1,Reason_2,Reason_3,Reason_4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [28]:
column_names_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Date', 'Transportation Expense', 
                          'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
                           'Children', 'Pets', 'Absenteeism Time in Hours']

df = df[column_names_reordered]
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


### Create a Checkpoint

In [34]:
df_reason_mod = df.copy()

### 'Date'

In [35]:
type(df_reason_mod['Date'][0])

str

In [38]:
df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format = '%d/%m/%Y')
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2


#### Extract the Month Value:
Having month as its own column will allow us to check if employees tend to be more absent during certain months of the year.

In [39]:
df_reason_mod['Date'][0]

Timestamp('2015-07-07 00:00:00')

In [40]:
df_reason_mod['Date'][0].month

7

In [42]:
list_months = []
numRows = df_reason_mod.shape[0]
numRows

700

In [43]:
for i in range(numRows):
    list_months.append(df_reason_mod['Date'][i].month)
    
df_reason_mod['Month'] = list_months
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7


#### Extract the Day of the Week:
Having day as its own column will allow us to check if employees tend to be more absent during certain day of the week. We will represent the days from Monday through Sunday numerically as 0 to 6.

In [63]:
def date_to_weekday(date_value):
    return date_value.weekday()

df_reason_mod['Day of the Week'] = df_reason_mod['Date'].apply(date_to_weekday)
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month,Day of the Week
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7,3


In [64]:
# Drop date column and reorder columns
df_reason_mod = df_reason_mod.drop(['Date'], axis = 1)
column_names_time_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month', 'Day of the Week', 
                               'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 
                               'Body Mass Index', 'Education', 'Children', 'Pets', 'Absenteeism Time in Hours']

df_reason_mod = df_reason_mod[column_names_time_reordered]
df_reason_date_mod = df_reason_mod.copy()
df_reason_date_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,1,2,1,2


### 'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index'
Let's dicuss some of the other columns in this dataframe:
- Travel expenses (monthly) = fuel + parking + meals + transportation + other => as this increases, it would make sense for someone to be less inclined to come to work
- Distance to work => as this increases, it would make sense for someone to be less inclined to come to work
- Age => how old someone is can always have an impact on someone's behavior
- Daily Work Load average = average amount of time spent working per day, in min => this could have an impact on whether or not an employee will come into the office or not to complete their work
- Body Mass Index = indicator for an under, normal, overweight, or even obese person, which could correlate to a medical condition that keeps the person away from work

### 'Education', 'Children', 'Pets'
All of these columns represent categorical data. 'Children' and 'Pets' indicate how many children and pets the person has precisely, so these columns can be left alone. However, with 'Education', the numbers do not have numeric meaning, so it needs to be transformed into a dummy variable.

In [65]:
df_reason_date_mod['Education'].unique()

array([1, 3, 2, 4])

1 = high school, 2 = graduate, 3 = post-graduate, 4 = master or doctor

In [66]:
df_reason_date_mod['Education'].value_counts()

1    583
3     73
2     40
4      4
Name: Education, dtype: int64

Separating between graduate, post-graduate, and master's or doctor degrees becomes less relevant for this study, as most of the datapoints are high school anyway. Therefore, we can combined 2-4 into a single category.

In [67]:
df_reason_date_mod['Education'] = df_reason_date_mod['Education'].map({1:0, 2:1, 3:1, 4:1})
df_reason_date_mod['Education'].value_counts()

0    583
1    117
Name: Education, dtype: int64

### Final Checkpoint

In [69]:
data_preprocessed = df_reason_date_mod.copy()
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


## Create a logistic regression to predict absenteeism
Now that we have initially preprocessed our data, we can begin preparing our data for a logistic regression model. First, we need to decide how we want to classify our inputs.

## Create the targets
We want to classify our inputs as either moderately absent or excessively absent. We will take the median value of 'Absenteeism Time in Hours' and use it as a cut-off line. Everything below the line is moderately absent and everything above is excessively absent. In this way, the dataset will be implicitly balanced (there will be a roughly equal number of 0s and 1s for the logistic regression).

Alternatively, if we had more data, we could have found other ways to deal with the issue. For instance, we could have assigned some arbitrary value as a cut-off line, instead of the median.

In [71]:
# Find the median of 'Absenteeism Time in Hours'
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

Note that what this line does is to assign 1 to anyone who has been absent 4 hours or more (more than 3 hours). That is the equivalent of taking half a day off.

In [87]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)
targets

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [76]:
# Places the targets series into the original df
data_preprocessed['Excessive Absenteeism'] = targets
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


We want to check if the dataset is balanced (what % of targets are 1s).

In [77]:
# targets.sum() => number of 1s that there are
# targets.shape[0] => length of the targets array
targets.sum() / targets.shape[0]

0.45571428571428574

Therefore, ~45.6% of targets are 1s, which is within a 45-55 split, so the dataset is balanced enough for the logistic regression.

In [127]:
# Create a checkpoint by dropping the unnecessary variables
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis=1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


## Select the inputs for the regression

In [128]:
# Create a variable that will contain the inputs (everything without the targets)
unscaled_inputs = data_with_targets.iloc[:,:-1]

## Standardize the inputs
Since data of different magnitude (scale) can be biased towards high values, we want all inputs to be of similar magnitude.

It's important that we not standardize all inputs, but only the ones we choose. For example, we would not want to standardize the Reasons 1-4 columns as they have dummy values of 0 or 1. Standardizing them would result in losing the interpretability of their values.

In [129]:
# Import the libraries needed to create a Custom Scaler class based on StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

# Define scaler as an empty object that will contain the scaling information for this particular dataset
absenteeism_scaler = StandardScaler()

absenteeism_scaler will be used to subtract the mean and divide by the standard deviation variablewise (featurewise).

In [130]:
# Create the Custom Scaler class
class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # Decide what information we need to declare a CustomScaler object and what is calculated/declared as we do
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        # s caler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # The fit method, which again is based on StandardScaler
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # The transform method, which does the actual scaling
    def transform(self, X, y=None, copy=None):
        # Record the initial order of the columns
        init_col_order = X.columns
        # Scale all features that we chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        # Declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        # Return a data frame which contains all scaled features and all 'not scaled' features
        # Use the original order (that we recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [131]:
# Choose the columns to scale
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [132]:
# Select the columns to omit
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

# Create the columns to scale, based on the columns to omit
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]
columns_to_scale

['Month',
 'Day of the Week',
 'Transportation Expense',
 'Distance to Work',
 'Age',
 'Daily Work Load Average',
 'Body Mass Index',
 'Children',
 'Pets']

In [133]:
# Declare a scaler object, specifying the columns we want to scale
absenteeism_scaler = CustomScaler(columns_to_scale)

In [134]:
# Fit the data (calculate mean and standard deviation); they are automatically stored inside the object 
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month', 'Day of the Week', 'Transportation Expense',
                      'Distance to Work', 'Age', 'Daily Work Load Average',
                      'Body Mass Index', 'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [135]:
# Actually standardize the data using parameters previously found
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.007725,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,-0.007725,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,0.668253,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.668253,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,1,-0.919030,-0.589690


We can see that our dummy variables were left untouched, as we wanted.

In [136]:
# Check the shape of the inputs
scaled_inputs.shape

(700, 14)

## Split the data into train & test and shuffle
We want to remove any types of dependencies that come from order of the dataset.

### Import the relevant module

In [137]:
# Import train_test_split so we can split our data into train and test
from sklearn.model_selection import train_test_split

### Split

In [138]:
# Split the data 
train_test_split(scaled_inputs, targets)

# declare 4 variables for the split
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [139]:
# Check the shape of the train inputs and targets
print (x_train.shape, y_train.shape)

(560, 14) (560,)


In [140]:
# Check the shape of the test inputs and targets
print (x_test.shape, y_test.shape)

(140, 14) (140,)


## Logistic regression with sklearn

In [141]:
# Import the LogReg model from sklearn
from sklearn.linear_model import LogisticRegression

# Import the 'metrics' module, which includes important metrics we may want to use
from sklearn import metrics

### Training the model

In [142]:
# Create a logistic regression object
reg = LogisticRegression()

In [143]:
# Fit our train inputs
reg.fit(x_train,y_train)

/Users/Sujay/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [144]:
# Assess the train accuracy of the model
reg.score(x_train,y_train)

0.7714285714285715

Based on the training data used, the model learned to classify ~78% of the observations correctly.

### Manually check the accuracy

In [146]:
# Find the model outputs according to our model
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [147]:
# Compare them with the targets
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [148]:
# ACTUALLY compare the two variables
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [149]:
# Find out in how many instances we predicted correctly
np.sum((model_outputs==y_train))

432

In [150]:
# Get the total number of instances
model_outputs.shape[0]

560

In [151]:
# Calculate the accuracy of the model
np.sum((model_outputs==y_train)) / model_outputs.shape[0]

0.7714285714285715

### Finding the intercept and coefficients

In [152]:
# Get the intercept (bias) of our model
reg.intercept_

array([-1.47082548])

In [153]:
# Get the coefficients (weights) of our model
reg.coef_

array([[ 2.62614283,  0.84432   ,  2.93971875,  0.67771914,  0.16145652,
        -0.08257256,  0.60695978, -0.00741802, -0.16862442, -0.00383568,
         0.26749958, -0.23085763,  0.35572764, -0.28559431]])

In [154]:
# Save the names of the columns in an ad-hoc variable
feature_name = unscaled_inputs.columns.values

# Use the coefficients from this table (they will be exported later and will be used in Tableau)
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,Coefficient
0,Reason_1,2.626143
1,Reason_2,0.844320
2,Reason_3,2.939719
3,Reason_4,0.677719
4,Month,0.161457
5,Day of the Week,-0.082573
6,Transportation Expense,0.606960
7,Distance to Work,-0.007418
8,Age,-0.168624
9,Daily Work Load Average,-0.003836


Move the intercept to the top of the summary table.

In [155]:
# Move all indices by 1
summary_table.index = summary_table.index + 1

# Add the intercept at index 0
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

# Sort the df by index
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.470825
1,Reason_1,2.626143
2,Reason_2,0.844320
3,Reason_3,2.939719
4,Reason_4,0.677719
5,Month,0.161457
6,Day of the Week,-0.082573
7,Transportation Expense,0.606960
8,Distance to Work,-0.007418
9,Age,-0.168624


## Interpreting the coefficients

Since the coefficients are standardized, we can determine their relatively important based off of their size.

Since this is a logistic regression, the coefficients we are predicting are the log(odds). To make them more interpretable, we want to find the odds ratio of each feature by taking the exponential of the coefficients.

An odds ratio of 1 would mean that for a unit change in the standardized feature, the odds increase by a multiple equal to the odds ratio (1 = no change).

In [156]:
# Create a new Series to show the odds ratio of each feature
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-1.470825,0.229736
1,Reason_1,2.626143,13.820360
2,Reason_2,0.844320,2.326395
3,Reason_3,2.939719,18.910527
4,Reason_4,0.677719,1.969381
5,Month,0.161457,1.175221
6,Day of the Week,-0.082573,0.920745
7,Transportation Expense,0.606960,1.834845
8,Distance to Work,-0.007418,0.992609
9,Age,-0.168624,0.844826


In [157]:
# Sort the table according to odds ratio
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
3,Reason_3,2.939719,18.910527
1,Reason_1,2.626143,13.820360
2,Reason_2,0.844320,2.326395
4,Reason_4,0.677719,1.969381
7,Transportation Expense,0.606960,1.834845
13,Children,0.355728,1.427219
11,Body Mass Index,0.267500,1.306693
5,Month,0.161457,1.175221
10,Daily Work Load Average,-0.003836,0.996172
8,Distance to Work,-0.007418,0.992609


- Features with coefficient close to 0 and odds ratio close to 1 do not seem to make a different given the rest of the features.
    - Daily Work Load Average
    - Distance to Work
    - Day of the Week
- The four reasons for absence seem to be the most important predictors (in order from 1 to 4 are poisoning, various diseases, pregnancy and giving birth, and light diseases). We can easily understand these coefficients based on these reasons. If you're poisoned, you definitely won't come in to work. If you are sick, you will stay home. If you have a gynecologist appointment, you may still come in to work, but if you're giving birth or having pregnancy problems, you will not.  
- Overall, when a person has stated an explicit reason, we have a much higher chance of getting excessive absence, which makes sense.
- Higher transportation expense, having children, and higher body mass index are all strongly positively correlated to excessive absence, which makes sense based on our previous predictions.
- Having pets and having higher education seem to be strongly negatively correlated with excessive absence. If you have several pets, it's possible/likely that you have someone else taking care of them while you are at work. If you're more educated, you can likely make a greater contribution to your work and therefore you will be more likely to be there more often.

## Testing the model

In [158]:
# Assess the test accuracy of the model
reg.score(x_test,y_test)

0.7428571428571429

Based on data that the model has never seen before, in 74.3% of the cases, the model will predict (correctly) if the person is going to be excessively absent. The test accuracy is only slightly lower than the train accuracy, so the model is not significanlty overfitted.

Now let's find the predicted probabilities of each class.

In [162]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.73685721, 0.26314279],
       [0.60918629, 0.39081371],
       [0.41367898, 0.58632102],
       [0.80123548, 0.19876452],
       [0.07360132, 0.92639868],
       [0.31482473, 0.68517527],
       [0.30996677, 0.69003323],
       [0.13143709, 0.86856291],
       [0.79426475, 0.20573525],
       [0.75019913, 0.24980087],
       [0.48367988, 0.51632012],
       [0.19799007, 0.80200993],
       [0.07753484, 0.92246516],
       [0.70995519, 0.29004481],
       [0.30224948, 0.69775052],
       [0.57267672, 0.42732328],
       [0.54083385, 0.45916615],
       [0.57389761, 0.42610239],
       [0.38014204, 0.61985796],
       [0.04814556, 0.95185444],
       [0.69716193, 0.30283807],
       [0.79219655, 0.20780345],
       [0.39173151, 0.60826849],
       [0.4186287 , 0.5813713 ],
       [0.25818169, 0.74181831],
       [0.75527615, 0.24472385],
       [0.51057945, 0.48942055],
       [0.86794852, 0.13205148],
       [0.19785515, 0.80214485],
       [0.78285796, 0.21714204],
       [0.

The first column shows the probability of a particular observation to be 0, while the second one - to be 1.

In [160]:
predicted_proba.shape

(140, 2)

In [161]:
# Select ONLY the probabilities referring to 1s
predicted_proba[:,1]

array([0.26314279, 0.39081371, 0.58632102, 0.19876452, 0.92639868,
       0.68517527, 0.69003323, 0.86856291, 0.20573525, 0.24980087,
       0.51632012, 0.80200993, 0.92246516, 0.29004481, 0.69775052,
       0.42732328, 0.45916615, 0.42610239, 0.61985796, 0.95185444,
       0.30283807, 0.20780345, 0.60826849, 0.5813713 , 0.74181831,
       0.24472385, 0.48942055, 0.13205148, 0.80214485, 0.21714204,
       0.3749067 , 0.68956837, 0.68976236, 0.53913229, 0.20780345,
       0.50790295, 0.21408487, 0.75084196, 0.43502114, 0.58927018,
       0.22777885, 0.43722182, 0.21924622, 0.43428676, 0.81295852,
       0.58244563, 0.69724136, 0.27338655, 0.20552251, 0.18317411,
       0.59131998, 0.38115576, 0.67045219, 0.28704924, 0.8497776 ,
       0.46817998, 0.89190316, 0.25870233, 0.35673457, 0.35514834,
       0.72095696, 0.6618861 , 0.31310986, 0.79285261, 0.20025819,
       0.26743725, 0.09841081, 0.2328088 , 0.73526565, 0.32952965,
       0.2134669 , 0.32994943, 0.90821459, 0.43626886, 0.61791

The logistic regression model determined that probabilities below 0.5 are 0 and those above 0.5 are 1, but here we can see how close to 0 and 1 they actually are.

## Save the model

In [163]:
# Import the relevant module
import pickle

In [164]:
# Pickle the model file
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [165]:
# Pickle the scaler file so we can standardize future data in the same way we did here
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)